In [1]:
import math
import pandas as pd 
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt

import h3 # h3 bins from uber

## Loading helper functions

These functions have been copied over from notebooks that already exist for this project. Ideally, these would be imported from a module.

In [2]:
#Loading the data
def create_crash_df(train_file = '../Inputs/Train.csv'):  
    crash_df = pd.read_csv(train_file, parse_dates=['datetime'])
    return crash_df

#Creating temporal features like months, weekdays, etc.
def create_temporal_features(df):
    dict_windows = {1: "00-03", 2: "03-06", 3: "06-09", 4: "09-12", 5: "12-15", 6: "15-18", 7: "18-21", 8: "21-24"}
    dict_months = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
               7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}
    df["time_window"] = df["datetime"].apply(lambda x: math.floor(x.hour / 3) + 1)
    df["time_window_str"] = df["time_window"].apply(lambda x: dict_windows.get(x))
    df["day"] = df["datetime"].apply(lambda x: x.day)
    df["month"] = df["datetime"].apply(lambda x: dict_months.get(x.month))
    df["year"] = df["datetime"].apply(lambda x: x.year)
    df["weekday"] = df["datetime"].apply(lambda x: x.weekday())
    return df

#Exporting the dataframe back to csv
def export_df_to_csv(df,path_file='../Inputs/train_h3.csv'):
    df.to_csv(path_file,index=False)
    print(f'file created {path_file}') 

#Joins provided data sets for road segments into one file, indicating latitude and longitude for each segment. This enables placement in h3 bins.    
def join_segment_files(path='../Inputs/', road_surveys='Segment_info.csv',segments_geometry='segments_geometry.geojson'):
    ''' 
        Load the survey data, Load the segment geometry, Join the two segment dfs.
        return a combined dataframe
    '''
    road_surveys = pd.read_csv(path+road_surveys)
    road_segment_locs = gpd.read_file(path+segments_geometry)
    segments_merged = pd.merge(road_segment_locs, road_surveys, on='segment_id', how='left')
    segments_merged["longitude"] = segments_merged.geometry.centroid.x
    segments_merged["latitude"] = segments_merged.geometry.centroid.y
    segments_merged = assign_hexbin(segments_merged)
    return segments_merged

#Defines time clusters, see docstring for more info
def assign_TW_cluster(weekday, time_window, holiday=0, strategy='baseline'):
    '''
    Can be used in a lambda function to return the time window cluster for a given day and time window.
    e.g. crash_df["cluster"] = crash_df.apply(lambda x: return_TW_cluster(x.weekday, x.time_window_str) ,axis=1)
    This is called by the function: create_cluster_feature.
    '''
    if strategy == 'baseline':
        return 'baseline'
    
    if strategy == 'mean_shift_modified':
        if weekday == 7:
            return 'off_peak'        
        elif weekday == 6:
            return 'off_peak'
        elif weekday in [0,1,2,3,4]:
            if time_window in ["06-09"]:
                return 'peak'
            elif time_window in ["09-12", "12-15", "15-18", "18-21"]:
                return 'middle'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'    
        elif weekday == 5:
            if time_window in ["06-09", "12-15", "15-18", "18-21"]:
                return 'middle'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'
            elif time_window in ["09-12"]:
                return 'peak'
    
    elif strategy == 'saturday_2':
        if weekday == 7:
            return 'off_peak'        
        elif weekday == 6:
            return 'off_peak'
        elif weekday in [0,1,2,3,4]:
            if time_window in ["06-09"]:
                return 'peak'
            elif time_window in ["09-12", "12-15", "15-18", "18-21"]:
                return 'middle'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'    
        elif weekday == 5:
            if time_window in ["06-09", "12-15", "15-18", "18-21"]:
                return 'saturday_busy'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'
            elif time_window in ["09-12"]:
                return 'saturday_busy'    
    
    elif strategy == 'holiday_7':
        if weekday == 7:
            return 'holiday'        
        elif weekday == 6:
            return 'sunday'
        elif weekday in [0,1,2,3,4]:
            if time_window in ["06-09"]:
                return 'peak'
            elif time_window in ["09-12", "12-15", "15-18", "18-21"]:
                return 'middle'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'    
        elif weekday == 5:
            if time_window in ["06-09", "12-15", "15-18", "18-21"]:
                return 'saturday_busy'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'
            elif time_window in ["09-12"]:
                return 'saturday_busy'      

    elif strategy == 'holiday_7':
        if weekday == 7:
            return 'holiday'        
        elif weekday == 6:
            return 'sunday'
        elif weekday in [0,1,2,3,4]:
            if time_window in ["06-09"]:
                return 'peak'
            elif time_window in ["09-12", "12-15", "15-18", "18-21"]:
                return 'middle'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'    
        elif weekday == 5:
            if time_window in ["06-09", "12-15", "15-18", "18-21"]:
                return 'saturday_busy'
            elif time_window in ["00-03", "03-06", "21-24"]:
                return 'off_peak'
            elif time_window in ["09-12"]:
                return 'saturday_busy'      
    

    elif strategy == 'no_cluster':
        return (str(weekday)+str(time_window)+str(holiday))

#Creates time cluster feature in existing data frame
def create_cluster_feature(crash_df, strategy='baseline', verbose=0):
    '''
    Function takes crash df and creates new column with tw cluster labels.
    If verbose is increased, the time window clusters will be visualised.
    '''
    crash_df["cluster"] = crash_df.apply(lambda x: 
                                         assign_TW_cluster(weekday=x.weekday,
                                                           time_window=x.time_window_str,
                                                           strategy=strategy) 
                                         ,axis=1)
    
    print(f'{crash_df.cluster.nunique()} clusters created')
    if verbose > 1:
        tb_clusters = sns.FacetGrid(crash_df,hue='cluster', height=5)
        tb_clusters.map(sns.stripplot,'weekday', 'time_window_str', s=20, 
                                       order = ['00-03', '03-06', '06-09', '09-12', 
                                                '12-15', '15-18', '18-21', '21-24'],
                                    label = 'Time Window Clusters')
    return crash_df

# Prediction models

Some text

## A. Identifying frequency outliers

Take historical data but cut off "frequency outliers" which occurred only once in the whole data set -> [provide function to generate list of frequency outlier hex bins].

**New helper function**

In [3]:
def assign_hexbin(df,lat_column="latitude",lon_column="longitude", hexbin_resolution=6):
    '''Assigning hex bins based on h3 classification and latitude and longitude'''
    df["h3_zone_{}".format(hexbin_resolution)] = df.apply(lambda x: h3.geo_to_h3(x[lat_column], x[lon_column], hexbin_resolution),axis=1)
    return df

In [4]:
df_raw = create_crash_df()
df = create_temporal_features(df_raw)
df = assign_hexbin(df)

In [5]:
df_cluster = create_cluster_feature(df, strategy='mean_shift_modified', verbose=0)

3 clusters created


**New helper function**

In [6]:
def rta_per_cluster_and_bins(df_cluster):
    '''Add up RTA's per hex bin and time custer'''
    df_rta = df_cluster.groupby([df_cluster.columns[-2], "cluster"]).agg({"uid": "count"}).reset_index()
    col_names = [df_rta.columns[0]] + [df_rta.columns[1]] + ["RTA"]
    df_rta.columns = col_names
    return df_rta

In [7]:
df_rta = rta_per_cluster_and_bins(df_cluster)

**New helper function**

In [8]:
def get_list_of_h3(df_hex, df_clusters):
    '''Create list of unique hex bins times unique time clusters'''
    return list(set(df_hex[df_hex.columns[0]])) * df_clusters["cluster"].nunique()

In [9]:
hex_cluster_comb = get_list_of_h3(df_rta, df_cluster)

**New helper function**

In [10]:
def get_list_of_states(df_clusters, df_hex):
    '''Create list of time clusters'''
    states = []
    for state in df_clusters["cluster"].unique():
        states += ([state] * df_hex[df_hex.columns[0]].nunique())
    return states

In [11]:
states = get_list_of_states(df_cluster, df_rta)

**New helper function**

In [12]:
def create_empty_df(list_hex_bins, list_time_clusters):
    '''Create an empty data frame of format hex bins * time clusters'''
    df_empty = pd.DataFrame(data=[list_hex_bins, list_time_clusters]).T
    df_empty.columns = [df_rta.columns[0], "cluster"]
    return df_empty

In [13]:
df_empty = create_empty_df(hex_cluster_comb, states)

**New helper function**

In [14]:
def fill_df_hex_rta(df_empty, df_rta):
    '''Join road traffic accidents onto empty data frame'''
    df_merged = pd.merge(df_empty, df_rta, on=[df_empty.columns[0], df_empty.columns[1]], how="outer")
    df_filled = df_merged.fillna(0)
    df_filled = df_filled.sort_values(by=[df_empty.columns[0], df_empty.columns[1]])
    return df_filled

In [15]:
df_filled = fill_df_hex_rta(df_empty, df_rta)

**New helper function**

In [16]:
def create_raw_pred_input_df(df_hex_bins):
    '''Based on hex bin resolution creates an empty data frame for each 3 hour time window for each hex bin.
     This results in a n * 2 dataframe (columns: time_windows, hex_bins) where number of rows equals hex_bins * 4369.
     4369 is the result of days between start and end date (in days) * 8 time windows per day (24 / 3 hours)'''
    #Create dataframe to get the accurate amount of 3-hour time windows for the desired time frame
    date_start = '2018-01-01'
    date_end = '2019-07-01'
    dates = pd.date_range(date_start, date_end, freq='3h')
    all_days_df = pd.DataFrame(dates, columns=["dates"])

    time_windows = list(all_days_df["dates"])
    len_windows = all_days_df.shape[0]
    list_unique_hexbins = df_hex_bins[df_hex_bins.columns[0]].unique()
    
    list_bins_per_window = []
    list_time_windows = []
    
    for i in range(0, len(list_unique_hexbins)):
        list_bins_per_window += len_windows * [list_unique_hexbins[i]]
        list_time_windows += time_windows
        
    input_df = {"time_windows": list_time_windows, "hex_bins": list_bins_per_window}
    df_pred_template = pd.DataFrame(data=input_df)
    
    return df_pred_template

In [17]:
df_raw_pred = create_raw_pred_input_df(df_filled)

In [18]:
df_raw_pred["time_window_key"] = df_raw_pred["time_windows"].apply(lambda x: str(x.year) + "-" + str(x.month) + "-" + str(x.day) + "-" + str(math.floor(x.hour / 3)))
df_cluster["time_window_key"] = df_cluster["datetime"].apply(lambda x: str(x.year) + "-" + str(x.month) + "-" + str(x.day) + "-" + str(math.floor(x.hour / 3)))

**New helper function**

In [19]:
def rta_per_time_window(df_cluster):
    '''Add up RTA's per time window'''
    df_tw = df_cluster.groupby(["time_window_key", "h3_zone_6"]).agg({"uid": "count"}).reset_index()
    col_names = ["time_window_key"] + ["hex_bins"] + ["RTA"]
    df_tw.columns = col_names
    return df_tw

In [20]:
df_tw = rta_per_time_window(df_cluster)

**New helper function**

In [21]:
def fill_overall_df(df_raw_pred, df_rta_per_tw):
    '''Join road traffic accidents onto empty data frame that consists of time windows (8 per day) for all days (1.5 years) for all hex bins. 
    For combinations with no accidents, NaNs will be converted into 0.'''
    df_merged = pd.merge(df_raw_pred, df_rta_per_tw, on=["time_window_key", "hex_bins"], how="outer")
    df_merged = df_merged.fillna(0)
    return df_merged

In [22]:
df_raw_filled = fill_overall_df(df_raw_pred, df_tw)

list_of_c = list(df_raw_filled.columns)
list_of_c[0] = "datetime"
df_raw_filled.columns = list_of_c

In [23]:
df_raw_filled = create_temporal_features(df_raw_filled)

In [24]:
df_final = create_cluster_feature(df_raw_filled, strategy='mean_shift_modified', verbose=0)

3 clusters created


In [25]:
df_classes = df_final.groupby("hex_bins")
df_classes = df_classes.agg({'RTA': [np.mean, np.std, np.sum, np.count_nonzero]})
df_classes = df_classes.reset_index()

In [26]:
df_classes.columns = ["hex_bins", "RTA_mean", "RTA_std", "RTA_sum", "RTA_nonzero"]

In [27]:
df_freq_outliers = df_classes.loc[df_classes["RTA_nonzero"] == 1]

In [29]:
list_freq_outliers = df_freq_outliers["hex_bins"].values
#list(list_freq_outliers)

## B. Using RTA frequency as a prediction measure

For each hex bin, use the frequencies (sum of occurrences, not the magnitude) for each time window as a prediction value -> [provide function to generate data frame of 56 (3 hour) time windows for each hex bin].

In [32]:
df_final.head()

,datetime,hex_bins,time_window_key,RTA,time_window,time_window_str,day,month,year,weekday,cluster
0,2018-01-01 00:00:00,867a44a6fffffff,2018-1-1-0,0.0,1,00-03,1,Jan,2018,0,off_peak
1,2018-01-01 03:00:00,867a44a6fffffff,2018-1-1-1,0.0,2,03-06,1,Jan,2018,0,off_peak
2,2018-01-01 06:00:00,867a44a6fffffff,2018-1-1-2,0.0,3,06-09,1,Jan,2018,0,peak
3,2018-01-01 09:00:00,867a44a6fffffff,2018-1-1-3,0.0,4,09-12,1,Jan,2018,0,middle
4,2018-01-01 12:00:00,867a44a6fffffff,2018-1-1-4,0.0,5,12-15,1,Jan,2018,0,middle


In [67]:
df_freq = df_final.groupby(["hex_bins", "weekday", "time_window_str"])
df_freq = df_freq.agg({'RTA': [np.mean, np.std, np.sum, np.count_nonzero]})
df_freq = df_freq.reset_index()

In [68]:
117 * 7 * 8

6552

In [69]:
#df_freq.head(56)

In [70]:
df_freq.shape

(6552, 7)

In [71]:
df_freq_filtered = df_freq.loc[~df_freq["hex_bins"].isin(list_of_bins)]

In [72]:
df_freq_filtered.shape
df_freq_filtered.columns = ["hex_bins", "weekday", "time_window", "RTA_mean", "RTA_std", "RTA_sum", "RTA_freq"]

In [73]:
df_freq_filtered.head()

,hex_bins,weekday,time_window,RTA_mean,RTA_std,RTA_sum,RTA_freq
0,867a44a6fffffff,0,00-03,0.0,0.0,0.0,0.0
1,867a44a6fffffff,0,03-06,0.0,0.0,0.0,0.0
2,867a44a6fffffff,0,06-09,0.0,0.0,0.0,0.0
3,867a44a6fffffff,0,09-12,0.0,0.0,0.0,0.0
4,867a44a6fffffff,0,12-15,0.0,0.0,0.0,0.0


In [74]:
df_freq_filtered.describe()

,weekday,RTA_mean,RTA_std,RTA_sum,RTA_freq
count,4032.000000,4032.000000,4032.000000,4032.000000,4032.000000
mean,3.000000,0.019934,0.089871,1.555060,1.048363
std,2.000248,0.060241,0.219307,4.699287,2.694370
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,0.012821,0.113228,1.000000,1.000000
max,6.000000,0.846154,4.463192,66.000000,37.000000


In [57]:
#df_freq_filtered.groupby("hex_bins").RTA_freq.sum().sort_values(ascending=False)

In [58]:
export_df_to_csv(df_freq,path_file='../Inputs/predictions_freq.csv')

file created ../Inputs/predictions_freq.csv


In [59]:
export_df_to_csv(df_freq_filtered,path_file='../Inputs/predictions_freq_h3_filtered.csv')

file created ../Inputs/predictions_freq_h3_filtered.csv


## C. Using weather data to predict RTA occurrence (yes/no?) per time window and hex_bin class

Add weather data (data per time window) to B, also fit regression model on weather data for two different "classes" of hex bins (dangerous, not so dangerous) [provide code that can output a probability of there being an accident (classification) for each hex bin in each time window (Note: These will still be the same for all hex bins of a class)].

## D. Using weather data and traffic speed data to predict RTA occurrence (yes/no?) per time window and hex_bin

Extend C to also include data that is specific per time window and hex bin. This allows the regression model to output a different value for each time window and hex bin (not only hex bin class).

****

## Test Area

In [39]:
list_of_bins = ['867a45067ffffff',
 '867a45077ffffff',
 '867a4511fffffff',
 '867a4512fffffff',
 '867a45147ffffff',
 '867a4515fffffff',
 '867a45177ffffff',
 '867a45817ffffff',
 '867a4584fffffff',
 '867a4585fffffff',
 '867a458dfffffff',
 '867a458f7ffffff',
 '867a45a8fffffff',
 '867a45b0fffffff',
 '867a45b17ffffff',
 '867a45b67ffffff',
 '867a45b77ffffff',
 '867a6141fffffff',
 '867a614d7ffffff',
 '867a616b7ffffff',
 '867a6304fffffff',
 '867a632a7ffffff',
 '867a63307ffffff',
 '867a6331fffffff',
 '867a6360fffffff',
 '867a63667ffffff',
 '867a6396fffffff',
 '867a656c7ffffff',
 '867a65797ffffff',
 '867a6e18fffffff',
 '867a6e1b7ffffff',
 '867a6e4c7ffffff',
 '867a6e517ffffff',
 '867a6e59fffffff',
 '867a6e5a7ffffff',
 '867a6e5b7ffffff',
 '867a6e657ffffff',
 '867a6e737ffffff',
 '867a6e797ffffff',
 '867a6e79fffffff',
 '867a6e7b7ffffff',
 '867a6ecf7ffffff',
 '867a6ed47ffffff',
 '867a6ed97ffffff','867a6eda7ffffff']

In [ ]:
df = pd.DataFrame(list_of_bins)

In [ ]:
export_df_to_csv(df,path_file='../Inputs/h3_excluded.csv')